In [1]:
import dask.dataframe as dd
import pandas as pd
import numpy as np
import os
from datetime import datetime, timedelta
import time
import json
import ast
from tqdm import tqdm
import requests
import warnings
warnings.filterwarnings('ignore') # turn off warnings

import psycopg2


connUAT = psycopg2.connect(
     host='localhost',
     database='uat',
     port=5431,
     user='postgres',
     password='1234567890'
  )

connPROC = psycopg2.connect(
     host='localhost',
     database='prod',
     port=5431,
     user='postgres',
     password='1234567890'
  )

cursorUAT = connUAT.cursor()
cursorPROC = connPROC.cursor()

def selectdf(query,conn):
    df = pd.read_sql(query, conn)
    return df
  
def disconnect(cursor,conn):
    try:
        if cursor:
            cursor.close()
        if conn:
            conn.close()
        print("Disconnected from database.")
    except Exception as e:
        print(f"Lỗi khi đóng kết nối: {e}")   
  
def SQL_push(sql_table_name,python_table,conn,cursor,inplace):
    python_table.replace([np.nan], [None],inplace=True)
    
    if inplace == True:
        try:
            sql_query = f'truncate table {sql_table_name}'
            cursor.execute(sql_query)
            conn.commit()
        except:
            pass
            
    sql = f"INSERT INTO {sql_table_name} VALUES ({','.join(['%s'] * len(python_table.columns))})"

    data = [tuple(row) for _, row in tqdm(python_table.iterrows(), total=len(python_table), desc="Preparing data")]

    # Chèn dữ liệu với xử lý lỗi
    for row in tqdm(data, total=len(data), desc="Inserting rows"):
        try:
            cursor.execute(sql, row)
        except Exception as e:
            print(f"Lỗi khi chèn dòng {row}: {e}")
            conn.rollback()  # Hủy giao dịch khi có lỗi, tránh PostgreSQL khóa transaction

    # Sau khi xong thì commit lại
    connUAT.commit()
    print("PUSH DATA: DONE")

pathfolder = r'F:\Work\Caohoc_2024_2026\PTPM_project\ml-latest'

In [2]:
disconnect(cursorUAT,connUAT)

Disconnected from database.


## Creadit

In [2]:
q = '''
    with 
        metadata as
            (
                select distinct *
                from staging.stg_tmdb_json_movie_metadata stjmm 
                where request_json = '{"success": false, "status_code": 34, "status_message": "The resource you requested could not be found."}'
                order by id		
            )
    select distinct *
    from staging.stg_tmdb_json_movie_credits stjmc 
    where id not in (select distinct id from metadata)
        and request_json <> '{"success": false, "status_code": 34, "status_message": "The resource you requested could not be found."}'
    order by id 
    '''
creadit = selectdf(q,connUAT)
creadit

,id,request_json
0,2,"{""id"": 2, ""cast"": [{""adult"": false, ""gender"": ..."
1,3,"{""id"": 3, ""cast"": [{""adult"": false, ""gender"": ..."
2,5,"{""id"": 5, ""cast"": [{""adult"": false, ""gender"": ..."
3,6,"{""id"": 6, ""cast"": [{""adult"": false, ""gender"": ..."
4,11,"{""id"": 11, ""cast"": [{""adult"": false, ""gender"":..."
...,...,...
37048,100313,"{""id"": 100313, ""cast"": [{""adult"": false, ""gend..."
37049,100318,"{""id"": 100318, ""cast"": [{""adult"": false, ""gend..."
37050,100323,"{""id"": 100323, ""cast"": [{""adult"": false, ""gend..."
37051,100327,"{""id"": 100327, ""cast"": [{""adult"": false, ""gend..."


In [3]:
cast_crew_list = []

for i in tqdm((set(creadit['id']))):
    datastr = creadit[creadit['id']==i]['request_json'].values[0]
    data = json.loads(f'''{datastr}''')
    cast_crew_list.append(data)
   
creaditdf = pd.DataFrame(cast_crew_list)
creaditdf.sort_values(by='id',inplace=True)
creaditdf

  0%|          | 0/37053 [00:00<?, ?it/s]

100%|██████████| 37053/37053 [00:13<00:00, 2738.47it/s]


,id,cast,crew
0,2,"[{'adult': False, 'gender': 2, 'id': 54768, 'k...","[{'adult': False, 'gender': 2, 'id': 16767, 'k..."
1,3,"[{'adult': False, 'gender': 2, 'id': 4826, 'kn...","[{'adult': False, 'gender': 1, 'id': 54766, 'k..."
2,5,"[{'adult': False, 'gender': 2, 'id': 3129, 'kn...","[{'adult': False, 'gender': 0, 'id': 3116, 'kn..."
3,6,"[{'adult': False, 'gender': 2, 'id': 2880, 'kn...","[{'adult': False, 'gender': 2, 'id': 2042, 'kn..."
4,11,"[{'adult': False, 'gender': 2, 'id': 2, 'known...","[{'adult': False, 'gender': 2, 'id': 1, 'known..."
...,...,...,...
37048,100313,"[{'adult': False, 'gender': 2, 'id': 31771, 'k...","[{'adult': False, 'gender': 2, 'id': 930633, '..."
37049,100318,"[{'adult': False, 'gender': 2, 'id': 33923, 'k...","[{'adult': False, 'gender': 2, 'id': 551674, '..."
37050,100323,"[{'adult': False, 'gender': 1, 'id': 121038, '...","[{'adult': False, 'gender': 2, 'id': 138172, '..."
37051,100327,"[{'adult': False, 'gender': 2, 'id': 31771, 'k...","[{'adult': False, 'gender': 2, 'id': 551674, '..."


## Metadata

In [ ]:
sql_check = '''
            select distinct *
            from staging.stg_tmdb_json_movie_metadata stjmm 
            where request_json <> '{"success": false, "status_code": 34, "status_message": "The resource you requested could not be found."}'
            order by id
            '''
movie_json = selectdf(sql_check,connUAT)
# list_c['id'] = list_c['id'].astype(str)  # Đảm bảo cùng kiểu dữ liệu
movie_json.sort_values(by='id',inplace=True)
movie_json

data_dict_list = []

for i in tqdm(set(movie_json['id'])):
    data_str = movie_json[movie_json['id']==i]['request_json'].values[0]
    data = f'''{data_str}'''
    data_dict = json.loads(data)
    data_dict_list.append(data_dict)
    
movie_metadata = pd.DataFrame(data_dict_list)
movie_metadata.sort_values(by='id',inplace=True)
movie_metadata


# movie_metadata2 = movie_metadata[['id','title','original_title','original_language','release_date','status','overview','tagline','adult','popularity','homepage','poster_path','runtime','budget','revenue','vote_average','vote_count']]
# movie_metadata2.head(5)

,id,request_json
0,2,"{""adult"": false, ""backdrop_path"": ""/hQ4pYsIbP2..."
1,2,"{""adult"": false, ""backdrop_path"": ""/hQ4pYsIbP2..."
2,3,"{""adult"": false, ""backdrop_path"": ""/l94l89eMmF..."
3,3,"{""adult"": false, ""backdrop_path"": ""/l94l89eMmF..."
4,5,"{""adult"": false, ""backdrop_path"": ""/f2t4JbUvQI..."
...,...,...
85459,1144338,"{""adult"": false, ""backdrop_path"": null, ""belon..."
85460,1145358,"{""adult"": false, ""backdrop_path"": ""/4OIJmgez12..."
85461,1146246,"{""adult"": false, ""backdrop_path"": ""/t86SkVd35W..."
85462,1146249,"{""adult"": false, ""backdrop_path"": ""/eNopYAzvtf..."


100%|██████████| 85304/85304 [00:26<00:00, 3259.63it/s]


,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,imdb_id,origin_country,original_language,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
1,False,/hQ4pYsIbP22TMXOUdSfC2mjWrO0.jpg,"{'id': 1382526, 'name': 'Aki Kaurismäki's Prol...",0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",None,2,tt0094675,[FI],fi,...,1988-10-21,0,73,"[{'english_name': 'Finnish', 'iso_639_1': 'fi'...",Released,None,Ariel,False,7.129,345
2,False,/l94l89eMmFKh7na2a1u5q67VgNx.jpg,"{'id': 1382526, 'name': 'Aki Kaurismäki's Prol...",0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",None,3,tt0092149,[FI],fi,...,1986-10-17,0,74,"[{'english_name': 'Finnish', 'iso_639_1': 'fi'...",Released,None,Shadows in Paradise,False,7.306,407
3,False,/f2t4JbUvQIjUF5FstG1zZFAp02N.jpg,None,4000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 80, 'nam...",https://www.miramax.com/movie/four-rooms/,5,tt0113101,[US],en,...,1995-12-09,4257354,98,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Twelve outrageous guests. Four scandalous requ...,Four Rooms,False,5.900,2683
4,False,/iUUpKunmBN5l8goObADBaFHnxQ8.jpg,None,21000000,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",None,6,tt0107286,[US],en,...,1993-10-15,12136938,109,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Don't move. Don't whisper. Don't even breathe.,Judgment Night,False,6.500,346
5,False,/2w4xG178RpB4MDAIfTkqAuSJzec.jpg,"{'id': 10, 'name': 'Star Wars Collection', 'po...",11000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",http://www.starwars.com/films/star-wars-episod...,11,tt0076759,[US],en,...,1977-05-25,775398007,121,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,"A long time ago in a galaxy far, far away...",Star Wars,False,8.203,21056
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49780,False,None,None,0,[],None,1144338,None,[],en,...,None,0,0,[],Canceled,None,Table,False,0.000,0
50074,False,/4OIJmgez12DITjHbWiNf1C7uG1d.jpg,None,0,"[{'id': 37, 'name': 'Western'}]",None,1145358,tt23469212,[US],en,...,2023-07-07,0,96,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Play your cards right and you just might survive.,Dead Man's Hand,False,5.778,9
50369,False,/t86SkVd35W9COtCGtbxxb3UoQpW.jpg,None,0,"[{'id': 27, 'name': 'Horror'}, {'id': 53, 'nam...",None,1146246,tt6810618,[CA],en,...,2024-08-22,0,84,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,None,The Deep Web: Murdershow,False,4.600,5
50371,False,/eNopYAzvtfvaj1yCI0AeVAOlHzU.jpg,None,0,"[{'id': 27, 'name': 'Horror'}, {'id': 53, 'nam...",None,1146249,tt17519798,[US],en,...,2023-07-12,0,88,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Insane in the membrane.,Cabin Girl,False,5.100,19


In [7]:
movie_metadata.columns.to_list()

['adult',
 'backdrop_path',
 'belongs_to_collection',
 'budget',
 'genres',
 'homepage',
 'id',
 'imdb_id',
 'origin_country',
 'original_language',
 'original_title',
 'overview',
 'popularity',
 'poster_path',
 'production_companies',
 'production_countries',
 'release_date',
 'revenue',
 'runtime',
 'spoken_languages',
 'status',
 'tagline',
 'title',
 'video',
 'vote_average',
 'vote_count']

In [24]:
# movie_metadata2.columns.to_list()

In [25]:
# movie_metadata[['id','belongs_to_collection']]

In [ ]:
# # DONE
# movies = pd.read_csv(os.path.join(pathfolder,'movies.csv'))
# movies1 = movies[['movieId','genres']]
# max_col = [len(i) for i in list(movies1['genres'].str.split('|'))]
# movies_genres = pd.DataFrame()

# for i in range(max(max_col)):
#     data = movies.copy()
#     data['genres'] = data['genres'].str.split('|',expand=True)[i]
#     data = data[data['genres'].isnull()==False]
#     movies_genres = pd.concat([movies_genres,data])

# movies_genres.sort_values(by='movieId',ascending=True,inplace=True)
# del movies_genres['title']
# movies_genres

## Links

In [35]:
# movies = pd.read_csv(os.path.join(pathfolder,'movies.csv'))
links = pd.read_csv(os.path.join(pathfolder,'links.csv'))
links = links[links['tmdbId'].isnull()==False]
links.sort_values(by='tmdbId',inplace=True)
links


,movieId,imdbId,tmdbId
4366,4470,94675,2.0
12656,61724,92149,3.0
17,18,113101,5.0
474,479,107286,6.0
257,260,76759,11.0
...,...,...,...
86365,288447,28212579,1144338.0
86458,288733,23469212,1145358.0
86466,288753,6810618,1146246.0
86513,288913,17519798,1146249.0


,movieId,genres
0,1,Adventure
0,1,Fantasy
0,1,Animation
0,1,Comedy
0,1,Children
...,...,...
86533,288971,Horror
86534,288975,Documentary
86535,288977,Thriller
86535,288977,Crime


## keyword

In [43]:
# DONE -- CSV
moives_tags = pd.read_csv(os.path.join(pathfolder,'tags.csv'))
moives_tags['date_tag'] = pd.to_datetime(moives_tags['timestamp'],unit='s')
moives_tags['tag'] = moives_tags['tag'].str.lower().str.strip()
moives_tags['tag'] = moives_tags['tag'].replace('',np.nan)
moives_tags.rename(columns={'tag':'keywords'},inplace=True)
moives_tags['keyid'] = moives_tags['userId'].astype('int').astype('str') + moives_tags['movieId'].astype('int').astype('str') + moives_tags['timestamp'].astype('int').astype('str')
moives_tags = moives_tags[['keyid','userId','movieId','keywords','date_tag']]
moives_tags

,keyid,userId,movieId,keywords,date_tag
0,102601430666558,10,260,good vs evil,2015-05-03 15:22:38
1,102601430666505,10,260,harrison ford,2015-05-03 15:21:45
2,102601430666538,10,260,sci-fi,2015-05-03 15:22:18
3,1412211311600756,14,1221,al pacino,2011-07-25 13:32:36
4,1412211311600746,14,1221,mafia,2011-07-25 13:32:26
...,...,...,...,...,...
2328310,3309231765991507547491,330923,176599,politically correct,2017-10-09 11:11:31
2328311,33093333171351279384,330933,3317,coming of age,2012-10-26 19:23:04
2328312,33093333171351279389,330933,3317,sexuality,2012-10-26 19:23:09
2328313,33094757821154110902,330947,5782,not luc besson,2006-07-28 18:21:42


In [8]:
# moives_tags_agg = moives_tags[['movieId','tag']].drop_duplicates()
# moives_tags_agg = moives_tags_agg.groupby('movieId').agg(keywords=('tag',list)).reset_index()
# moives_tags_agg['keywords'] = moives_tags_agg['keywords'].astype('str')
# moives_tags_agg

In [9]:
# test = moives_tags[moives_tags['movieId']==260][['movieId','tag']]
# test.drop_duplicates(inplace=True)
# test[test['tag'].str.contains('jedi')]

In [10]:
# moives_tags.isnull().sum()

## Rating

In [ ]:
# DONE
movies_ratings = pd.read_csv(os.path.join(pathfolder,'ratings.csv'))
movies_ratings['date_time'] = pd.to_datetime(movies_ratings['timestamp'],unit='s')
del movies_ratings['timestamp']
movies_ratings

,userId,movieId,rating,date_time
0,1,1,4.0,2008-11-03 17:52:19
1,1,110,4.0,2008-11-05 06:04:46
2,1,158,4.0,2008-11-03 17:31:43
3,1,260,4.5,2008-11-03 18:00:04
4,1,356,5.0,2008-11-03 17:58:39
...,...,...,...,...
33832157,330975,8340,2.0,2004-08-04 01:34:16
33832158,330975,8493,2.5,2004-08-04 02:15:09
33832159,330975,8622,4.0,2004-08-04 01:09:37
33832160,330975,8665,3.0,2004-08-04 01:09:25


In [ ]:
# movies_ratings.isnull().sum()

userId       0
movieId      0
rating       0
date_time    0
dtype: int64

In [ ]:
# connUAT.commit()
# connUAT.close()